In [7]:
using Distributed
using LinearAlgebra

In [8]:
M = Matrix{Float64}[rand(1000,1000) for i = 1:10];

In [12]:
S = pmap(svdvals, M);

In [11]:
function hankel(y::AbstractArray)
    m, time_duration = size(y) # m - dimention of output vector y, time_duration - length of timeseries (number of time steps)
    q = Int(round(time_duration/2)) # q - is the size of Hankel matrix 
    H = zeros(eltype(y), q * m , q) 
    for r = 1:q, c = 1:q # r - rows, c -columns
        H[(r-1)*m+1:r*m, c] = y[:, r+c-1]
    end
    return H, m
end

function lsid_n_ACx0(Y::AbstractArray, Δt, n) 
    # y - output time series dim[y] = m x number_of_time_steps
    # n - rank of the system we want to identify
    
    H, m = hankel(Y) # Hankel matrix and dimention of output (should be 12 in our case)
    U, Σ, Vd = svd(H) # Singular value decomposition of H to U,  Σ,  V†
    
    s = Diagonal(sqrt.(Σ)) # Matrix square root 
    U = U * s
    Vd = s * Vd
      
    C = U[1:m, 1:n] # m -dimention of output, n - rank of the system
    
    U_up = U[1:end-m, :] # U↑
    U_down = U[m+1:end, :] # U↓
    
    A = pinv(U_up) * U_down
    # Ac = log(A)/Δt 
    # Ac = Ac[1:n, 1:n] 
    A = A[1:n, 1:n] # n - estimated rank of the system
    
    x0 = pinv(U) * H
    x0 = x0[1:n, 1]
    
    return A, C, x0 # was A, Ac, C, x0

end

10-element Vector{Vector{Float64}}:
 [500.40144865321827, 18.05787339639904, 18.00591633050705, 17.98704232021531, 17.967302413579052, 17.83076269405649, 17.687424921197728, 17.64503102086061, 17.603992781144797, 17.575869064917477  …  0.1262660325708624, 0.10994159268410109, 0.1015014844786633, 0.09703744483043734, 0.07821303088640014, 0.05718120490734678, 0.04201186031656555, 0.02954839234648307, 0.020572591124782665, 0.007520069855714909]
 [500.35650645387807, 18.10182729871007, 18.02579972258705, 17.93853268465495, 17.856244234485203, 17.77424005592659, 17.699300328296918, 17.661744701669242, 17.641289273414163, 17.57849633785355  …  0.12957672806927137, 0.11933680729827839, 0.10052057335771254, 0.08142434189840721, 0.07266203616652622, 0.0654503043034533, 0.047542446545930095, 0.03531445445477708, 0.019433214244410968, 0.007348972990488419]
 [500.11167694996794, 18.260960361215066, 17.987221697798102, 17.92183187822627, 17.900908499267423, 17.85447000738299, 17.747893377073485, 17

In [13]:
a = zeros(10)

Threads.@threads for i = 1:10
           a[i] = Threads.threadid()
       end
#julia --threads auto


In [14]:
a

10-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0